In [ ]:
import pandas
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from yellowbrick.text import TSNEVisualizer

## Load in Data

In [ ]:
DF_FL0 = pandas.read_excel('/Users/alice.naghshineh/Desktop/michael_data/Florida.xlsx')

DF_Dependent = DF_FL0[['Q12PresidentialVote']].copy()

DF_Independent = DF_FL0[[
 'Party',
 'Issue',
 'Hispanic',
 'Race',
 'Marital Status',
 'Education',
 'Age',
 'Gender',
 'Raceparty',
 'Genderparty']]

DF_FL0 = DF_FL0.drop(['Q8Clinton', 'Q9Trump'], axis=1)

In [ ]:
DF_FL0.head(10)

In [ ]:
DF_FL0.describe()

## Visualize Feature Counts (Raw & Normalized)

In [ ]:
candidates = ('Clinton', 'Trump', 'Johnson', 'Stein', 'Undecided')
y_pos = np.arange(len(candidates))
counts = DF_FL0['Q12PresidentialVote'].value_counts().tolist()

plt.bar(y_pos, counts, align='center', alpha=0.5)
plt.xticks(y_pos, candidates)
plt.ylabel('Counts')
plt.title('Votes per Candidate')

plt.show()

In [ ]:
def get_counts_percent(column):
    c = DF_FL0[column].value_counts()
    p = DF_FL0[column].value_counts(normalize=True)
    t = pandas.concat([c,p], axis=1, keys=['counts', '%'])
    print('\n{}:\n{}'.format(column,t))

In [ ]:
for column in DF_Independent:
    get_counts_percent(column)

In [ ]:
for column in ['Marital Status','Age','Gender']:
    categories = DF_FL0[column].unique().tolist()
    y_pos = np.arange(len(categories))
    counts = DF_FL0[column].value_counts().tolist()

    plt.bar(y_pos, counts, align='center', alpha=0.5)
    plt.xticks(y_pos, categories)
    plt.ylabel('Counts')
    plt.title('{}'.format(column))

    plt.show()

In [ ]:
ax = sns.countplot(y='Party', hue='Race', data=DF_FL0)

In [ ]:
ax = sns.countplot(y='Q12PresidentialVote', hue='Race', data=DF_FL0)

In [ ]:
ax = sns.countplot(y='Race', hue='Issue', data=DF_FL0)
plt.legend(loc='lower right');

## Prepare Categorical Variables for Modeling

In [ ]:
DF_Independent_dummies = pandas.get_dummies(DF_Independent)

le = preprocessing.LabelEncoder()

le.fit(DF_Dependent['Q12PresidentialVote'])

target = le.transform(DF_Dependent['Q12PresidentialVote'])

In [ ]:
[x_train, x_test, y_train, y_test] = train_test_split(DF_Independent_dummies, target, test_size=0.2)

## Try KNeighbors

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 1).fit(DF_Independent_dummies, target) 
  
# accuracy on X_test 
accuracyknn = knn.score(DF_Independent_dummies, target) 
print(accuracyknn) 

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 1).fit(x_train, y_train)
predicted = knn.predict(x_test)
print("{}: {}".format(knn.__class__.__name__, f1_score(y_test, predicted, average='micro')))

## ... and a Clustering Visualization

In [ ]:
tsne = TSNEVisualizer(size=(1080, 720))
tsne.fit(DF_Independent_dummies, target)

print(tsne.poof())